In [1]:
###################################################################################################
# Define parameters and query data
###################################################################################################

# Use this to read in packages from other directories ---------
import sys, os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))) # two dirs back
#--------------------------------------------------------------
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stndata as stndata

variable = 'mint' # Tmin (Daily Low Temperature)

# Location and bounding box
location_name = 'St. Louis, MO'
nlat = 38.9     # northern latitude
slat = 38.5     # southern latitude
wlon = -90.55   # western longitude
elon = -90.17   # eastern longitude

# Query variable
var, meta = stndata.bbox_multistn_daily(elem=variable,slat=slat,nlat=nlat,wlon=wlon,elon=elon,
                                        print_md=False)

mint: Reading in 14 total stations (station id: name, state) ...
#1. 112614: EAST ST LOUIS PARKS COLLEGE, IL
#2. 111160: CAHOKIA, IL
#3. 238791: WEBSTER GROVES, MO
#4. 237452: ST LOUIS SCIENCE CENTER, MO
#5. 93963: ST LOUIS EADS BRIDGE, MO
#6. 237465: ST LOUIS ST LOUIS UNIV, MO
#7. 237470: SAINT LOUIS WASHINGTON UNIV, MO
#8. 238525: UNIVERSITY CITY, MO
#9. 234272: JEFFERSON BARRACKS, MO
#10. 237398: ST CHARLES 7 SSW, MO
#11. 237462: SAINT LOUIS 12 NNW, MO
#12. 13994: ST LOUIS LAMBERT INTL AIRPORT, MO
#13. 93910: ST LOUIS LAMBERT, MO
#14. 237397: ST CHARLES ELM POINT, MO


In [2]:
# PLOT

# Use this to read in packages from other directories ---------
import sys, os 
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))) # two dirs back
#--------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy, cartopy.mpl.geoaxes, cartopy.io.img_tiles
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stnmeta as stnmeta
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stndata as stndata
from ClimateDataVisualizer.processing.bbox_dy import bbox_avg_dy
from ClimateDataVisualizer.processing.bbox_my import bbox_avg_my, bbox_max_my, bbox_min_my
from ClimateDataVisualizer.inset_axes.inset_axes import inset_map, inset_timeseries
import warnings

def annualcycle_tmin_plot(var,meta,location_name,nlat,slat,wlon,elon,syr,eyr,num_stn,iyr,minbuff,maxbuff,
                          majtick,mintick,incl_hist,incl_year,incl_info,incl_map,img_tile,lbl_buff,ext_buff):

    ############################################################################################################# 
    # Average data by day and year 
    ############################################################################################################# 

    # List of leap years from 1800 to 2100
    leapyears = [1800 + i * 4 for i in range((2100 - 1800) // 4 + 1)]
    isleap = False if iyr not in leapyears else True

    # Apply NaN filter to the dataframe based on minimum number of stations collecting data on a given day
    var_filt = var.copy()
    var_filt.loc[var_filt.iloc[:,1:].count(axis=1) < num_stn, var_filt.columns[1:]] = np.nan

    # Create dataframe of every day of the every year
    var_dy = bbox_avg_dy(var_filt,leap=isleap)

    ############################################################################################################# 
    # Auto-select dates based on parameters 
    ############################################################################################################# 

    # If earliest, take earliest year with data
    if syr == 'earliest':
        try:
           syr = int(np.min(var_dy.dropna(axis=1,how='all').columns[2:]))
        except:
           print('ERROR: There are no years with this many stations collecting data simultaneously.')

    # If latest, take latest year (but not this year) with fewer than 365 NaN values   
    if eyr == 'latest':
        for c in range(2,len(var_dy)+1):
            if var_dy[var_dy.columns[-c]].isna().sum() < 365:
                eyr = int(var_dy.columns[-c])
                break

    # Define back-end plotting year based on whether current year is a leap year or not
    plt_yr = 2020 if iyr in leapyears else 2022

    ############################################################################################################# 
    # Define figure 
    ############################################################################################################# 

    fig, ax = plt.subplots(figsize=[8,4],dpi=300)

    # Define x-axis time interval
    xtime = pd.date_range(start=str(plt_yr)+'-01-01', end=str(plt_yr)+'-12-31', freq='D')

    ############################################################################################################# 
    # PLOT HISTORICAL DATA 
    ############################################################################################################# 

    if incl_hist == True:

        # MEAN
        var_avg = np.nanmean(var_dy.iloc[:,var_dy.columns.get_loc(str(syr)):\
                                          var_dy.columns.get_loc(str(eyr))+1],axis=1)

        # 5th and 95th percentiles
        var_05 = np.nanpercentile(var_dy.iloc[:,var_dy.columns.get_loc(str(syr)):\
                                                var_dy.columns.get_loc(str(eyr))+1],5,axis=1)
        var_95 = np.nanpercentile(var_dy.iloc[:,var_dy.columns.get_loc(str(syr)):\
                                                var_dy.columns.get_loc(str(eyr))+1],95,axis=1)

        # Min and Max
        var_min = np.nanmin(var_dy.iloc[:,var_dy.columns.get_loc(str(syr)):\
                                                var_dy.columns.get_loc(str(eyr))+1],axis=1)
        var_max = np.nanmax(var_dy.iloc[:,var_dy.columns.get_loc(str(syr)):\
                                                var_dy.columns.get_loc(str(eyr))+1],axis=1)

        # Plot historical data
        ax.plot(xtime,var_avg,'-',c='k',lw=1.5,alpha=0.5,zorder=100)
        ax.fill_between(xtime,var_05,var_95,color='tab:blue',alpha=0.2,edgecolor=None,zorder=1)
        ax.fill_between(xtime,var_max,var_95,color='tab:blue',alpha=0.05,edgecolor=None,zorder=1)
        ax.fill_between(xtime,var_05,var_min,color='tab:blue',alpha=0.05,edgecolor=None,zorder=1)
        ax.plot(xtime,var_max,':',c='tab:blue',lw=0.25,alpha=0.75,zorder=10)
        ax.plot(xtime,var_min,':',c='tab:blue',lw=0.25,alpha=0.75,zorder=10)

        # Text indicators
        leg_hist = ax.legend([mpl.patches.Patch(facecolor='tab:blue', alpha=0.2, edgecolor=None)],
                             (r' $\bf{HISTORICAL \ DATA \ PERIOD :}$'+' {}-{}'.format(syr,eyr),''),
                             fontsize=8,framealpha=0.,bbox_to_anchor=(0,0,0.59,0.08))
        ax.add_artist(leg_hist)

        # How to read historical data
        leg = fig.add_axes([0.5, 0.2, 0.1, 0.16])
        leg.set_xticklabels([]), leg.set_xticks([]), leg.set_yticklabels([]), leg.set_yticks([]);
        randomness,mult,last,leg_var = 0.5+0.5*np.random.rand(10),[10,5,0.01,-5,-10],[9,5,0,-5,-9],np.zeros((5,10))
        for i in range(5):
            leg_var[i,:],leg_var[i,-1] = mult[i]*np.ones(10)*randomness, last[i]
        leg.plot(np.arange(10),leg_var[2,:],'-',c='k',lw=1.5,alpha=0.5,zorder=100)
        leg.fill_between(np.arange(10),leg_var[3,:],leg_var[1,:],color='tab:blue',alpha=0.2,edgecolor=None,zorder=1)
        leg.fill_between(np.arange(10),leg_var[4,:],leg_var[3,:],color='tab:blue',alpha=0.05,edgecolor=None,zorder=1)
        leg.fill_between(np.arange(10),leg_var[1,:],leg_var[0,:],color='tab:blue',alpha=0.05,edgecolor=None,zorder=1)
        leg.plot(np.arange(10),leg_var[0,:],':',c='tab:blue',lw=0.25,alpha=0.75,zorder=10)
        leg.plot(np.arange(10),leg_var[4,:],':',c='tab:blue',lw=0.25,alpha=0.75,zorder=10)
        tags = ['MAX','95$^{th}$','MEAN','5$^{th}$','MIN']
        for i in range(5):
            leg.text(10,leg_var[i,:][-1],tags[i],fontsize=6,alpha=0.5,weight='bold',ha='left',va='center')
        leg.set_title('How to read\nhistorical data',loc='center',fontsize=8,weight='bold',alpha=0.6);

    ############################################################################################################# 
    # PLOT INDIVIDUAL YEAR 
    ############################################################################################################# 

    if incl_year == True:
        ax.plot(pd.date_range(start=str(plt_yr)+'-01-01',end=str(plt_yr)+'-12-31',freq='D'),
                var_dy[str(iyr)],'-',c='b',lw=1,zorder=1000)
        leg_year = ax.legend([mpl.lines.Line2D([0],[0],c='tab:blue',lw=1.5)],(str(iyr),''),
                             framealpha=0.,bbox_to_anchor=(0,0,0.13,0.08),
                             prop={'weight': 'bold', 'size': 8})

    ############################################################################################################# 
    # Features to include? Can toggle on and off in parameters
    ############################################################################################################# 

    if incl_info == True:
        larger = (var_dy[str(iyr)] > var_avg).sum()
        smaller = (var_dy[str(iyr)] < var_avg).sum()
        ax.text(0.18,0.89,r'$\bf{'+str(iyr)+':}$ '+'\n'+r'$\bf{'+str(larger)+'}$'+' days above '+
                str(syr)+'-'+str(eyr)+' mean \n'+r'$\bf{'+str(smaller)+'}$'+' days below '+
                str(syr)+'-'+str(eyr)+' mean',ha='center',va='center',fontsize=8,transform=ax.transAxes)

    if incl_map == True:
        inset_map(ax=ax,meta=meta,var=var,width=0.7,height=0.8,markercolor='k',incl_year=incl_year,iyr=iyr,
                  iyr_col='tab:red',slat=slat,nlat=nlat,wlon=wlon,elon=elon,bbox_to_anchor=(0,0,1,0.97),
                  lbl_buff=lbl_buff,proj=cartopy.crs.PlateCarree(),ext_buff=ext_buff,img_tile=img_tile)

    ############################################################################################################# 
    # Title and text
    ############################################################################################################# 

    ax.text(0.75,0.02,r'$\bf{IMAGE:}$ Alex Thompson (@ajtclimate)'+'\n'+r'$\bf{DATA:}$'+
                   ' NOAA ACIS (http://data.rcc-acis.org)',ha='left',fontsize=5,transform=ax.transAxes)
    ax.set_title('Daily Low Temperature in '+location_name,loc='center',fontsize=14,pad=5,
                 weight='bold');

    ############################################################################################################# 
    # Axes specs 
    ############################################################################################################# 

    ax.set_xticks([str(plt_yr)+'-'+'{:02}'.format(month) for month in range(1, 13)])
    ax.set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
    ax.set_ylabel('Region Mean (°F)')
    ax.get_yaxis().set_major_locator(mpl.ticker.MultipleLocator(majtick))
    ax.get_yaxis().set_minor_locator(mpl.ticker.MultipleLocator(mintick))
    ax.grid(alpha=0.05)
    ax.set_xlim([pd.to_datetime(str(plt_yr-1)+'-12-30'),pd.to_datetime(str(plt_yr+1)+'-01-01')])
    ax.set_ylim([np.nanmin(var_min)-minbuff,np.nanmax(var_max)+maxbuff]);
    ax.spines[['right','top']].set_visible(False)

    return fig, var_dy, var_max, var_95, var_avg, var_05, var_min

In [3]:
# WIDGET

# Use this to read in packages from other directories ---------
import sys, os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))) # two dirs back
#--------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy, cartopy.mpl.geoaxes, cartopy.io.img_tiles
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stnmeta as stnmeta
from ClimateDataVisualizer.dataquery import NOAA_ACIS_stndata as stndata
from ClimateDataVisualizer.processing.bbox_dy import bbox_avg_dy
from ClimateDataVisualizer.inset_axes.inset_axes import inset_map, inset_timeseries
from ClimateDataVisualizer.interactives import plots
from ClimateDataVisualizer.downloads.file_options import pdf_opts, xcl_opts         
import ipywidgets as ipyw
from IPython.display import display, HTML

# URLs to use for user help guides, needs double quotes
url_annualcycle = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-annual-cycle"
url_timeseries  = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-time-series"
url_cumulative  = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-cumulative"
url_map         = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-map-of-stations"
url_yaxis       = "https://sites.google.com/view/ajtclimate/climate-data-viz/help-y-axis"

def annualcycle_tmin_widget(var,meta,location_name,nlat,slat,wlon,elon):

    #----------------------------------------------------------------------------------------------
    # Set up parameters to toggle
    #----------------------------------------------------------------------------------------------

    # Define years for historical range
    txt_hist = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_annualcycle} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Define years for historical range</a></h3>')
    text_hist = ipyw.Label(value='Include historical climate?',layout=ipyw.Layout(width='165px'))
    incl_hist = ipyw.Dropdown(options=[True,False],value=True,layout=ipyw.Layout(width='60px'))
    text_syr = ipyw.Label(value='Starting Year',layout=ipyw.Layout(width='125px'))
    text_eyr = ipyw.Label(value='Ending Year',layout=ipyw.Layout(width='125px'))
    text_stn = ipyw.Label(value='Minimum number of stations',layout=ipyw.Layout(width='175px'))
    syr = ipyw.Dropdown(options=['earliest',*np.sort(var['Date'].dt.year.unique())[:-1]],
                        value='earliest',layout=ipyw.Layout(width='100px'))
    eyr = ipyw.Dropdown(options=['latest',*np.sort(var['Date'].dt.year.unique())[::-1][1:]],
                        value='latest',layout=ipyw.Layout(width='100px'))
    if len(var.columns) == 2: # 'Date' and 1 data column
       num_stn = ipyw.Dropdown(options=[1],value=1,layout=ipyw.Layout(width='50px'))
    else:
       num_stn = ipyw.Dropdown(options=np.arange(1,len(var.columns)-1),value=1,
                               layout=ipyw.Layout(width='50px'))

    # Plot individual year
    txt_indv = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_annualcycle} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Plot individual year</a></h3>')
    text_year = ipyw.Label(value='Include individual year?',layout=ipyw.Layout(width='150px'))
    incl_year = ipyw.Dropdown(options=[True,False],value=True,layout=ipyw.Layout(width='70px'))
    text_iyr = ipyw.Label(value='Year Displayed',layout=ipyw.Layout(width='90px'))
    iyr = ipyw.Dropdown(options=var['Date'].dt.year.unique()[::-1],
                        value=var['Date'].dt.year.unique()[-1],layout=ipyw.Layout(width='70px'))

    # Map
    txt_map   = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_map} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Map of stations</a></h3>')
    text_map  = ipyw.Label(value='Include map?',layout=ipyw.Layout(width='100px'))
    incl_map  = ipyw.Dropdown(options=[True,False],value=True,layout=ipyw.Layout(width='80px'))
    text_lyr  = ipyw.Label(value='Image tile',layout=ipyw.Layout(width='100px'))
    img_tile  = ipyw.Dropdown(options=['QuadtreeTiles','GoogleTiles','OpenStreetMap','grey'],
                              value='QuadtreeTiles',layout=ipyw.Layout(width='80px'))
    text_lbl = ipyw.Label(value='Label distance',layout=ipyw.Layout(width='100px'))
    lbl_buff = ipyw.Dropdown(options=[('10%',0.1),('20%',0.2),('30%',0.3),('40%',0.4),('50%',0.5),
                                       ('60%',0.6),('70%',0.7),('80%',0.8),('90%',0.9),('None',1.5)],
                              value=0.3,layout=ipyw.Layout(width='80px'))
    text_ext = ipyw.Label(value='Lat/lon buffer',layout=ipyw.Layout(width='100px'))
    ext_buff = ipyw.Dropdown(options=[('0.1°',0.1),('0.25°',0.25),('0.5°',0.5),('1°',1.),('5°',5.)],
                              value=0.5,layout=ipyw.Layout(width='80px'))

    # Y-axis buffer
    txt_buff  = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_yaxis} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Y-axis buffer</a></h3>')
    maxb_desc = ipyw.Label(value='Buffer above y-axis',layout=ipyw.Layout(width='120px'))
    maxbuff   = ipyw.FloatText(value=20.,layout=ipyw.Layout(width='60px'))
    maxb_unit = ipyw.Label(value='°F')
    minb_desc = ipyw.Label(value='Buffer below y-axis',layout=ipyw.Layout(width='120px'))
    minbuff   = ipyw.FloatText(value=20.,layout=ipyw.Layout(width='60px'))
    minb_unit = ipyw.Label(value='°F')

    # Y-axis tick stride
    txt_tick   = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_yaxis} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Y-axis tick stride</a></h3>')
    majtk_desc = ipyw.Label(value='Major tick stride',layout=ipyw.Layout(width='120px'))
    majtick    = ipyw.FloatText(value=20.,layout=ipyw.Layout(width='60px'))
    majtk_unit = ipyw.Label(value='°F')
    mintk_desc = ipyw.Label(value='Minor tick stride',layout=ipyw.Layout(width='120px'))
    mintick    = ipyw.FloatText(value=5.,layout=ipyw.Layout(width='60px'))
    mintk_unit = ipyw.Label(value='°F')

    # Extras     
    txt_xtra  = ipyw.HTML(value=f'<h3><a style="color: black; "href={url_annualcycle} target="_blank" ' +
                                'onmouseover="this.style.textDecoration=\'underline\'" '+
                                'onmouseout="this.style.textDecoration=\'none\'"'+
                                '>Extras</a></h3>')
    text_info = ipyw.Label(value='Include info for year?',layout=ipyw.Layout(width='130px'))
    incl_info = ipyw.Dropdown(options=[True,False],value=True,layout=ipyw.Layout(width='70px'))

    #----------------------------------------------------------------------------------------------
    # Layout for dropdowns
    #----------------------------------------------------------------------------------------------

    ui = ipyw.VBox([
                    # FIRST ROW
                    ipyw.HBox([ipyw.VBox([
                                 # Define years for historical range
                                 ipyw.HBox([txt_hist],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([text_hist,incl_hist],layout=ipyw.Layout(
                                                                        justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([text_syr,syr]),
                                                       ipyw.HBox([text_eyr,eyr]),
                                                       ipyw.HBox([text_stn,num_stn])])],
                                           layout=ipyw.Layout(justify_content='center'))]),
                               ipyw.VBox([
                                 # Plot individual year
                                 ipyw.HBox([txt_indv],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([text_year,incl_year],
                                           layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([text_iyr,iyr],
                                           layout=ipyw.Layout(justify_content='space-between'))]),
                               ipyw.VBox([
                                 # Map 
                                 ipyw.HBox([txt_map],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([text_map,incl_map]),
                                                       ipyw.HBox([text_lyr,img_tile]),
                                                       ipyw.HBox([text_lbl,lbl_buff]),
                                                       ipyw.HBox([text_ext,ext_buff])])])])],
                               layout=ipyw.Layout(justify_content='space-around')),

                    # SECOND ROW
                    ipyw.HBox([ipyw.VBox([
                                 # Y-axis buffer
                                 ipyw.HBox([txt_buff],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([maxb_desc,maxbuff,maxb_unit]),
                                                       ipyw.HBox([minb_desc,minbuff,minb_unit])])],
                                           layout=ipyw.Layout(flex='1 1 auto',
                                                              justify_content='center'))]),
                               ipyw.VBox([
                                 # Y-axis tick stride
                                 ipyw.HBox([txt_tick],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([ipyw.VBox([ipyw.HBox([majtk_desc,majtick,majtk_unit]),
                                                       ipyw.HBox([mintk_desc,mintick,mintk_unit])])],
                                            layout=ipyw.Layout(flex='1 1 auto',
                                                               justify_content='space-between'))]),
                               ipyw.VBox([
                                 # Extras
                                 ipyw.HBox([txt_xtra],layout=ipyw.Layout(justify_content='center')),
                                 ipyw.HBox([text_info,incl_info],
                                           layout=ipyw.Layout(justify_content='center'))])],
                               layout=ipyw.Layout(justify_content='space-around'))
                     ])

    #----------------------------------------------------------------------------------------------
    # Display interactive plot
    #----------------------------------------------------------------------------------------------

    # Define function with only interactive components that runs widget.plot function
    def plot_fig(syr,eyr,num_stn,iyr,minbuff,maxbuff,majtick,mintick,incl_hist,incl_year,
                 incl_info,incl_map,img_tile,lbl_buff,ext_buff):
        fig,var_dy,var_max,var_95,var_avg,var_05,var_min = plots.annualcycle_tmin_plot(
                     var=var,meta=meta,location_name=location_name,nlat=float(nlat),
                     slat=float(slat),wlon=float(wlon),elon=float(elon),syr=syr,eyr=eyr,
                     num_stn=num_stn,iyr=iyr,minbuff=minbuff,maxbuff=maxbuff,majtick=majtick,
                     mintick=mintick,incl_hist=incl_hist,incl_year=incl_year,incl_info=incl_info,
                     incl_map=incl_map,img_tile=img_tile,lbl_buff=lbl_buff,ext_buff=ext_buff)

        # Create a button widget to download the figure as PDF
        pdf_output = ipyw.Output()
        def download_pdf(button):
           pdf_filename = 'figure.pdf'
           pdf_opts(fig=fig,pdf_output=pdf_output,pdf_filename=pdf_filename)
        pdf_but = ipyw.Button(description='Download Figure', layout={'width': '140px'})
        pdf_but.on_click(download_pdf)

        # Create a button widget to download data as Excel
        xcl_output = ipyw.Output()
        def download_xcl(button):
            with xcl_output: print('Download initiated. Can take more than a minute if over 100 stations '+
                                   'are queried. Do not click download button again.')
            xcl_filename = 'data.xlsx'
            with pd.ExcelWriter(xcl_filename) as w:
                var.assign(**{var.columns[0]: var.iloc[:,0].astype(str)}).to_excel(w,sheet_name='stations',index=False)
                meta.to_excel(w,sheet_name='metadata',index=False)
                if incl_hist == True:
                   pd.concat([pd.DataFrame({'month':var_dy['month'],'day':var_dy['day'],'max':var_max,'95th':var_95,
                                            'mean':var_avg,'5th':var_05,'min':var_min}),var_dy.iloc[:,2:]],axis=1
                             ).to_excel(w,sheet_name='historical',index=False)
                if incl_year == True:
                   pd.DataFrame({'month':var_dy['month'],'day':var_dy['day'],str(iyr):var_dy[str(iyr)]}
                                ).to_excel(w,sheet_name=str(iyr),index=False)
            xcl_opts(xcl_filename=xcl_filename,xcl_output=xcl_output)
        xcl_but = ipyw.Button(description='Download Data',layout=ipyw.Layout(width='140px'))
        xcl_but.on_click(download_xcl)

        # Display download buttons together
        button_box = ipyw.VBox([ipyw.HBox([pdf_but,xcl_but],layout=ipyw.Layout(justify_content='center'))])
        display(button_box,pdf_output,xcl_output)



    # Interactive output with only interactive components in dictionary
    out = ipyw.interactive_output(plot_fig,{'syr':syr,'eyr':eyr,'num_stn':num_stn,'iyr':iyr,
                                            'minbuff':minbuff,'maxbuff':maxbuff,'majtick':majtick,
                                            'mintick':mintick,'incl_hist':incl_hist,'incl_year':incl_year,
                                            'incl_info':incl_info,'incl_map':incl_map,'img_tile':img_tile,
                                            'lbl_buff':lbl_buff,'ext_buff':ext_buff})
    display(ui,out)

In [4]:
#-----------------
# Run function    
#-----------------

annualcycle_tmin_widget(var=var,meta=meta,location_name=location_name,nlat=nlat,slat=slat,wlon=wlon,elon=elon)


Output()